<a href="https://colab.research.google.com/github/stevejj4/Machine-Learning/blob/main/Insurance_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
from google.colab import auth  # Authorizing Google Colab
auth.authenticate_user()
import gspread  # Credentials for Google Sheets
from google.auth import default

creds, _ = default()
gc = gspread.authorize(creds)  # Authorizing the connection
import pandas as pd

# Open the Google Sheet by name
worksheet = gc.open('Insurancce_data').sheet1

# Get all values in list of rows
rows = worksheet.get_all_values()

# Create DataFrame and define columns
df = pd.DataFrame(rows)
df.columns = df.iloc[0]  # Set the first row as column names
df = df.iloc[1:]  # Remove the first row

In [ ]:
# Convert data types
df['Age'] = pd.to_numeric(df['Age'])
df['PremiumAmount'] = pd.to_numeric(df['PremiumAmount'])
df['PolicyStartDate'] = pd.to_datetime(df['PolicyStartDate'])
df['PolicyEndDate'] = pd.to_datetime(df['PolicyEndDate'])
